# Project Milestone 4
## Connecting to an API/Pulling in the Data and Cleaning/Formatting

### Problem Statement : Perform at least 5 data transformation and/or cleansing steps to your API data. For example:

1. Replace Headers
2. Format data into a more readable format
3. Identify outliers and bad data
4. Find duplicates
5. Fix casing or inconsistent values
6. Conduct Fuzzy Matching

You can submit a Jupyter Notebook or a PDF of your code. If you submit a .py file you need to also include a PDF or attachment of your results.


### Solution : 
Solution starts below

In [110]:
# As a first step, Since we do not get the weather data for all the cities.
# Will be using latititude and longitude to get the nearest city data
# To get the latitude and longitude, we will use the geopy
# We will filter out the city name, state name from the earlier df
# Add a column for latitude and longitude

In [12]:
import pandas as pd

header_ff = ["State Name", "PC No", "PC Name", "Electors","Male Vote Count","Female Vote Count", 
             "Third Gender Vote Count","Total EVM Votes Count","NRI Votes Count","Postal Votes Count","Total Votes Count", "Voter Turnout %",
            "Male Votes %", "Female Votes %","Third Gender Votes %"]
# Reading the flat file for city name and state name
df_ff = pd.read_excel("13. PC Wise Voters Turn Out.xls", skiprows=3,skipfooter=5,names=header_ff)
df_ff.head(10)

,State Name,PC No,PC Name,Electors,Male Vote Count,Female Vote Count,Third Gender Vote Count,Total EVM Votes Count,NRI Votes Count,Postal Votes Count,Total Votes Count,Voter Turnout %,Male Votes %,Female Votes %,Third Gender Votes %
0,Andaman & Nicobar Islands,1,Andaman & Nicobar Islands,318471,109407,97671,2,207080,0,318,207398,65.123041,64.633080,65.468402,22.222222
1,Andhra Pradesh,1,Aruku,1451418,523085,544258,19,1067362,0,10873,1078235,74.288386,73.705294,73.387125,20.212766
2,Andhra Pradesh,2,Srikakulam,1553860,548060,594937,14,1143011,0,21275,1164286,74.928629,70.079112,77.105128,6.698565
3,Andhra Pradesh,3,Vizianagaram,1503980,603059,605843,7,1208909,0,13524,1222433,81.279871,79.979603,80.796195,5.691057
4,Andhra Pradesh,4,Visakhapatnam,1829300,608306,624713,7,1233026,0,6895,1239921,67.781173,66.255537,68.569682,6.034483
5,Andhra Pradesh,5,Anakapalli,1521363,606111,626764,6,1232881,0,7575,1240456,81.535833,80.789163,81.286127,9.090909
6,Andhra Pradesh,6,Kakinada,1563930,616003,611330,66,1227399,0,9415,1236814,79.083719,79.378863,77.611861,29.333333
7,Andhra Pradesh,7,Amalapuram,1459556,617210,607337,0,1224547,0,10821,1235368,84.639986,84.623970,83.177365,0.000000
8,Andhra Pradesh,8,Rajahmundry,1534256,617706,626974,8,1244688,0,5800,1250488,81.504521,81.922786,80.368506,6.557377
9,Andhra Pradesh,9,Narsapuram,1439691,580912,587960,35,1168907,5,10255,1179162,81.903825,81.940001,80.477422,22.875817


In [34]:
# Need only State Name and PC Name Columns
df_api = df_ff[['State Name','PC Name']]
df_api

,State Name,PC Name
0,Andaman & Nicobar Islands,Andaman & Nicobar Islands
1,Andhra Pradesh,Aruku
2,Andhra Pradesh,Srikakulam
3,Andhra Pradesh,Vizianagaram
4,Andhra Pradesh,Visakhapatnam
...,...,...
537,West Bengal,Bardhaman Purba
538,West Bengal,Bardhaman Durgapur
539,West Bengal,Asansol
540,West Bengal,Bolpur


In [36]:
# combining city and state name and country as one string
# dfmi.loc[:, ('one', 'second')] 
df_api['Full Location'] = df_api.loc[:,('PC Name')] + ", " +df_api.loc[:,('State Name')] + ", India"
print(df_api.head(10))

                  State Name                    PC Name  \
0  Andaman & Nicobar Islands  Andaman & Nicobar Islands   
1             Andhra Pradesh                     Aruku    
2             Andhra Pradesh                 Srikakulam   
3             Andhra Pradesh               Vizianagaram   
4             Andhra Pradesh              Visakhapatnam   
5             Andhra Pradesh                 Anakapalli   
6             Andhra Pradesh                   Kakinada   
7             Andhra Pradesh                Amalapuram    
8             Andhra Pradesh                Rajahmundry   
9             Andhra Pradesh                 Narsapuram   

                                       Full Location  
0  Andaman & Nicobar Islands, Andaman & Nicobar I...  
1                      Aruku , Andhra Pradesh, India  
2                  Srikakulam, Andhra Pradesh, India  
3                Vizianagaram, Andhra Pradesh, India  
4               Visakhapatnam, Andhra Pradesh, India  
5                  A

<ipython-input-36-a172f54f25d0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_api['Full Location'] = df_api.loc[:,('PC Name')] + ", " +df_api.loc[:,('State Name')] + ", India"


In [48]:
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent="myGeocoder")

def getLatLong(loc):
    # format needed
    # lat=24&lon=87.58
    #Task 5: Fix casing or inconsistent values
    # Inconsistent Values have been taken care
    result = "Not Found"
    location = locator.geocode(loc)
    if(location != None):
        result = "lat={}&lon={}".format(location.latitude, location.longitude)
        # used beloe print for testing
        #print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))
    return result


df_api['LatLong'] = df_api['Full Location'].apply(lambda x: getLatLong(x))


<ipython-input-48-a62784544c25>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_api['LatLong'] = df_api['Full Location'].apply(lambda x: getLatLong(x))


In [75]:
# Reading Keys which are stored separate from github
# Reading for iqAir ----- NOT USING iqAIR due to limitation of calls
import urllib.request, urllib.parse, urllib.error
import json

with open("C:/git-bellevue/keys/airvisual-keys.json") as file:
   json_data = file.read()

data = json.loads(json_data)
key = data['keys']
serviceURL = "http://api.airvisual.com/v2/nearest_city?"
# Get key from properties - do not expose
# key = "&key=13b26ad6-1bc3-47f3-892d-db74944a46ee"

def getFinalURL(serviceURL,latlong,key):
    URL = serviceURL + latlong + "&key=" +key
    return URL


In [79]:
# openweather api setup
# Reading Keys which are stored separate from github
# Reading openweather.json file
import urllib.request, urllib.parse, urllib.error
import json

with open("C:/git-bellevue/keys/openweather-keys.json") as file:
   json_data = file.read()

data = json.loads(json_data)
key = data['key']
serviceURL = "http://api.openweathermap.org/data/2.5/air_pollution?"
#http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={API key}
# Get key from properties - do not expose
# &appid={API key}

def getOpenWeatherFinalURL(serviceURL,latlong,key):
    URL = serviceURL + latlong + "&appid="+key
    return URL

In [83]:
# This function calls api and returns json response
# Can make 5 calls per minute, so introducing sleep
# Can make 10000 calls in a month
# Now, will save the data in json file, so that dont need to download again

# Open weather has 
# 60 calls/minute
# 1,000,000 calls/month

import time

def getWeatherData(latlong):
    # Task 5: Fix casing or inconsistent values
    # Inconsistent Values have been taken care
    response = "Not Found"
    if(latlong != "Not Found"):
        # 1 secs or more needed - 60 calls a minute
        time.sleep(2)
        url = getOpenWeatherFinalURL(serviceURL,latlong,key)
        #print("Calling URL -",url)
        uh = urllib.request.urlopen(url)
        data = uh.read().decode()
        rsp_json = json.loads(data)
        return rsp_json
    else:
        return response
        
df_api['apiResponse'] = df_api['LatLong'].apply(lambda x: getWeatherData(x)) 

# Test Code
# df_api['LatLong'].apply(lambda x: getWeatherData(x))
# print(getWeatherData("lat=13.51111965&lon=92.91738841784462","test","test"))
# getWeatherData("Not Found","test","test")
# getting json response and saving it in a df column for processing later
# df_api['apiResponse'] = 
# openWeather - Test Result
# {'coord': {'lon': 92.9174, 'lat': 13.5111}, 'list': [{'main': {'aqi': 4}, 
# 'components': {'co': 390.53, 'no': 0, 'no2': 0.16, 'o3': 105.86, 'so2': 0.37, 
# 'pm2_5': 44.24, 'pm10': 49.83, 'nh3': 0.02}, 'dt': 1610398800}]}
# Test Transaction was success - iqAir
#print(getWeatherData("lat=13.51111965&lon=92.91738841784462"))
# Sample Response
#{'status': 'success', 'data': 
#{'city': 'Yangon', 'state': 'Yangon', 'country': 'Myanmar', 'location': 
# {'type': 'Point', 'coordinates': [96.15611, 16.80528]}, 
#'current': {'weather': {'ts': '2021-01-11T18:00:00.000Z', 
#'tp': 25, 'pr': 1012, 'hu': 88, 'ws': 2.57, 'wd': 210, 'ic': '02n'}, 
# 'pollution': {'ts': '2021-01-11T18:00:00.000Z', 'aqius': 112, 'mainus': 'p2', 'aqicn': 56, 'maincn': 'p2'}}}}
    

<ipython-input-83-8c22c0947f31>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_api['apiResponse'] = df_api['LatLong'].apply(lambda x: getWeatherData(x))


In [84]:
df_api

,State Name,PC Name,Full Location,LatLong,apiResponse
0,Andaman & Nicobar Islands,Andaman & Nicobar Islands,"Andaman & Nicobar Islands, Andaman & Nicobar I...",lat=13.51111965&lon=92.91738841784462,"{'coord': {'lon': 92.9174, 'lat': 13.5111}, 'l..."
1,Andhra Pradesh,Aruku,"Aruku , Andhra Pradesh, India",Not Found,Not Found
2,Andhra Pradesh,Srikakulam,"Srikakulam, Andhra Pradesh, India",lat=18.32002205&lon=83.91607719937166,"{'coord': {'lon': 83.9161, 'lat': 18.32}, 'lis..."
3,Andhra Pradesh,Vizianagaram,"Vizianagaram, Andhra Pradesh, India",lat=18.1120819&lon=83.4052196224888,"{'coord': {'lon': 83.4052, 'lat': 18.1121}, 'l..."
4,Andhra Pradesh,Visakhapatnam,"Visakhapatnam, Andhra Pradesh, India",lat=17.7231276&lon=83.3012842,"{'coord': {'lon': 83.3013, 'lat': 17.7231}, 'l..."
...,...,...,...,...,...
537,West Bengal,Bardhaman Purba,"Bardhaman Purba, West Bengal, India",lat=23.244051&lon=87.8689422,"{'coord': {'lon': 87.8689, 'lat': 23.2441}, 'l..."
538,West Bengal,Bardhaman Durgapur,"Bardhaman Durgapur, West Bengal, India",lat=23.4916613&lon=87.3181652,"{'coord': {'lon': 87.3182, 'lat': 23.4917}, 'l..."
539,West Bengal,Asansol,"Asansol, West Bengal, India",lat=23.6871297&lon=86.9746587,"{'coord': {'lon': 86.9747, 'lat': 23.6871}, 'l..."
540,West Bengal,Bolpur,"Bolpur, West Bengal, India",lat=23.661154&lon=87.6962424,"{'coord': {'lon': 87.6962, 'lat': 23.6612}, 'l..."


In [108]:
# Task 1: Format data into a more readable format
# below function will split json response to required attributes
# looking specifically polution data
# 'pollution': {'ts': '2021-01-11T18:00:00.000Z', 'aqius': 112, 'mainus': 'p2', 'aqicn': 56, 'maincn': 'p2'}}}}
    
#openweather - jsondata = '{"coord":[50,50],"list":[{"dt":1605182400,"main":{"aqi":1},"components":{"co":201.94053649902344,"no":0.01877197064459324,"no2":0.7711350917816162,"o3":68.66455078125,"so2":0.6407499313354492,"pm2_5":0.5,"pm10":0.540438711643219,"nh3":0.12369127571582794}}]}'
#iqAir - jsondata = '{"status":"success","data":{"city":"Durgapur","state":"West Bengal","country":"India","location":{"type":"Point","coordinates":[87.300171,23.530458]},"current":{"weather":{"ts":"2021-01-11T16:00:00.000Z","tp":21,"pr":1014,"hu":43,"ws":3.27,"wd":357,"ic":"03n"},"pollution":{"ts":"2021-01-11T14:00:00.000Z","aqius":166,"mainus":"p2","aqicn":112,"maincn":"p2"}}}}{"status":"success","data":{"city":"Durgapur","state":"West Bengal","country":"India","location":{"type":"Point","coordinates":[87.300171,23.530458]},"current":{"weather":{"ts":"2021-01-11T16:00:00.000Z","tp":21,"pr":1014,"hu":43,"ws":3.27,"wd":357,"ic":"03n"},"pollution":{"ts":"2021-01-11T14:00:00.000Z","aqius":166,"mainus":"p2","aqicn":112,"maincn":"p2"}}}}'
    
# print(splitJsonAQI(jsondata))

#Splitting/Extracting the AQI from json
def splitJsonAQI(json):
    if(json!="Not Found"):
        aqi = json['list'][0]['main']['aqi']
        return aqi
    else:
        return None
#Splitting/Extracting the CO from json    
def splitJsonCO(json):
    if(json!="Not Found"):
        aqi = json['list'][0]['components']['co']
        return aqi
    else:
        return None
#Splitting/Extracting the NO from json
def splitJsonNO(json):
    if(json!="Not Found"):
        aqi = json['list'][0]['components']['no']
        return aqi
    else:
        return None
#Splitting/Extracting the NO2 from json    
def splitJsonNO2(json):
    if(json!="Not Found"):
        aqi = json['list'][0]['components']['no2']
        return aqi
    else:
        return None
#Splitting/Extracting the SO2 from json
def splitJsonSO2(json):
    if(json!="Not Found"):
        aqi = json['list'][0]['components']['so2']
        return aqi
    else:
        return None
#Splitting/Extracting the O3 from json   
def splitJsonO3(json):
    if(json!="Not Found"):
        aqi = json['list'][0]['components']['o3']
        return aqi
    else:
        return None
#Splitting/Extracting the NH3 from json   
def splitJsonNH3(json):
    if(json!="Not Found"):
        aqi = json['list'][0]['components']['nh3']
        return aqi
    else:
        return None
#Splitting/Extracting the PM2.5 from json    
def splitJsonpm25(json):
    if(json!="Not Found"):
        aqi = json['list'][0]['components']['pm2_5']
        return aqi
    else:
        return None
#Splitting/Extracting the PM10 from json    
def splitJsonpm10(json):
    if(json!="Not Found"):
        aqi = json['list'][0]['components']['pm10']
        return aqi
    else:
        return None


df_api.loc[:,('aqi')] = df_api.loc[:,('apiResponse')].apply(lambda x: splitJsonAQI(x))
df_api.loc[:,('co')] = df_api.loc[:,('apiResponse')].apply(lambda x: splitJsonCO(x))
df_api.loc[:,('no')] = df_api.loc[:,('apiResponse')].apply(lambda x: splitJsonNO(x))
df_api.loc[:,('no2')] = df_api.loc[:,('apiResponse')].apply(lambda x: splitJsonNO2(x))
df_api.loc[:,('so2')] = df_api.loc[:,('apiResponse')].apply(lambda x: splitJsonSO2(x))
df_api.loc[:,('o3')] = df_api.loc[:,('apiResponse')].apply(lambda x: splitJsonO3(x))
df_api.loc[:,('nh3')] = df_api.loc[:,('apiResponse')].apply(lambda x: splitJsonNH3(x))
df_api.loc[:,('pm2_5')] = df_api.loc[:,('apiResponse')].apply(lambda x: splitJsonpm25(x))
df_api.loc[:,('pm10')] = df_api.loc[:,('apiResponse')].apply(lambda x: splitJsonpm10(x))


C:\softwares\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [109]:
df_api

,State Name,PC Name,Full Location,LatLong,apiResponse,aqi,co,no,no2,so2,o3,nh3,pm2_5,pm10
0,Andaman & Nicobar Islands,Andaman & Nicobar Islands,"Andaman & Nicobar Islands, Andaman & Nicobar I...",lat=13.51111965&lon=92.91738841784462,"{'coord': {'lon': 92.9174, 'lat': 13.5111}, 'l...",4.0,390.53,0.00,0.16,0.37,105.86,0.02,44.24,49.83
1,Andhra Pradesh,Aruku,"Aruku , Andhra Pradesh, India",Not Found,Not Found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Andhra Pradesh,Srikakulam,"Srikakulam, Andhra Pradesh, India",lat=18.32002205&lon=83.91607719937166,"{'coord': {'lon': 83.9161, 'lat': 18.32}, 'lis...",5.0,988.01,0.01,28.10,5.25,19.31,60.29,154.41,169.37
3,Andhra Pradesh,Vizianagaram,"Vizianagaram, Andhra Pradesh, India",lat=18.1120819&lon=83.4052196224888,"{'coord': {'lon': 83.4052, 'lat': 18.1121}, 'l...",5.0,961.30,6.15,14.91,3.19,0.03,15.20,145.56,167.78
4,Andhra Pradesh,Visakhapatnam,"Visakhapatnam, Andhra Pradesh, India",lat=17.7231276&lon=83.3012842,"{'coord': {'lon': 83.3013, 'lat': 17.7231}, 'l...",5.0,2216.34,135.90,38.39,64.85,0.00,2.31,240.73,268.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,West Bengal,Bardhaman Purba,"Bardhaman Purba, West Bengal, India",lat=23.244051&lon=87.8689422,"{'coord': {'lon': 87.8689, 'lat': 23.2441}, 'l...",5.0,988.01,0.00,16.45,2.03,24.68,12.29,74.45,88.31
538,West Bengal,Bardhaman Durgapur,"Bardhaman Durgapur, West Bengal, India",lat=23.4916613&lon=87.3181652,"{'coord': {'lon': 87.3182, 'lat': 23.4917}, 'l...",5.0,687.60,0.00,9.68,2.53,45.78,4.62,61.89,70.87
539,West Bengal,Asansol,"Asansol, West Bengal, India",lat=23.6871297&lon=86.9746587,"{'coord': {'lon': 86.9747, 'lat': 23.6871}, 'l...",5.0,634.19,0.00,7.97,1.70,49.35,3.74,60.24,69.58
540,West Bengal,Bolpur,"Bolpur, West Bengal, India",lat=23.661154&lon=87.6962424,"{'coord': {'lon': 87.6962, 'lat': 23.6612}, 'l...",5.0,741.01,0.00,10.37,1.34,41.13,6.02,68.20,78.22


In [113]:
# Task 2: Replace Headers
headers = ['State Name', 'PC Name', 'Full Location', 'LatLong', 'apiResponse',
       'AirQualityIndex', 'CarbonMonoxide', 'NitrouseOxide', 'NitrogenDioxide', 
           'SulphurDioxide', 'Ozone', 'Ammonia', 'PM2.5', 'PM10']
df_api.columns = headers
df_api

,State Name,PC Name,Full Location,LatLong,apiResponse,AirQualityIndex,CarbonMonoxide,NitrouseOxide,NitrogenDioxide,SulphurDioxide,Ozone,Ammonia,PM2.5,PM10
0,Andaman & Nicobar Islands,Andaman & Nicobar Islands,"Andaman & Nicobar Islands, Andaman & Nicobar I...",lat=13.51111965&lon=92.91738841784462,"{'coord': {'lon': 92.9174, 'lat': 13.5111}, 'l...",4.0,390.53,0.00,0.16,0.37,105.86,0.02,44.24,49.83
1,Andhra Pradesh,Aruku,"Aruku , Andhra Pradesh, India",Not Found,Not Found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Andhra Pradesh,Srikakulam,"Srikakulam, Andhra Pradesh, India",lat=18.32002205&lon=83.91607719937166,"{'coord': {'lon': 83.9161, 'lat': 18.32}, 'lis...",5.0,988.01,0.01,28.10,5.25,19.31,60.29,154.41,169.37
3,Andhra Pradesh,Vizianagaram,"Vizianagaram, Andhra Pradesh, India",lat=18.1120819&lon=83.4052196224888,"{'coord': {'lon': 83.4052, 'lat': 18.1121}, 'l...",5.0,961.30,6.15,14.91,3.19,0.03,15.20,145.56,167.78
4,Andhra Pradesh,Visakhapatnam,"Visakhapatnam, Andhra Pradesh, India",lat=17.7231276&lon=83.3012842,"{'coord': {'lon': 83.3013, 'lat': 17.7231}, 'l...",5.0,2216.34,135.90,38.39,64.85,0.00,2.31,240.73,268.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,West Bengal,Bardhaman Purba,"Bardhaman Purba, West Bengal, India",lat=23.244051&lon=87.8689422,"{'coord': {'lon': 87.8689, 'lat': 23.2441}, 'l...",5.0,988.01,0.00,16.45,2.03,24.68,12.29,74.45,88.31
538,West Bengal,Bardhaman Durgapur,"Bardhaman Durgapur, West Bengal, India",lat=23.4916613&lon=87.3181652,"{'coord': {'lon': 87.3182, 'lat': 23.4917}, 'l...",5.0,687.60,0.00,9.68,2.53,45.78,4.62,61.89,70.87
539,West Bengal,Asansol,"Asansol, West Bengal, India",lat=23.6871297&lon=86.9746587,"{'coord': {'lon': 86.9747, 'lat': 23.6871}, 'l...",5.0,634.19,0.00,7.97,1.70,49.35,3.74,60.24,69.58
540,West Bengal,Bolpur,"Bolpur, West Bengal, India",lat=23.661154&lon=87.6962424,"{'coord': {'lon': 87.6962, 'lat': 23.6612}, 'l...",5.0,741.01,0.00,10.37,1.34,41.13,6.02,68.20,78.22


In [115]:
# Task 3: Find if any duplicates
# Checking the Combination of PC Name and State Name
duplicateRowsDF = df_api[df_api.duplicated(['PC Name','State Name'])]
display(duplicateRowsDF)
# Since the Combination check returns no values we are good

,State Name,PC Name,Full Location,LatLong,apiResponse,AirQualityIndex,CarbonMonoxide,NitrouseOxide,NitrogenDioxide,SulphurDioxide,Ozone,Ammonia,PM2.5,PM10


In [120]:
# Task 4: Identify outliers and bad data
# Checking unique values for some of the pollution parameters
# AQI, Carbon Monoxide, PM2.5, PM10
# Additionally they should not be negative
print("AirQualityIndex-Max:",df_api['AirQualityIndex'].max())
print("AirQualityIndex-Min:",df_api['AirQualityIndex'].min())
print("Carbon Monoxide-Max:",df_api['CarbonMonoxide'].max())
print("Carbon Monoxide-Min:",df_api['CarbonMonoxide'].min())
print("PM2.5-Max:",df_api['PM2.5'].max())
print("PM2.5-Min:",df_api['PM2.5'].min())
print("PM10-Max:",df_api['PM10'].max())
print("PM10-Min:",df_api['PM10'].min())
# None of the values are negative, so should be good.

AirQualityIndex-Max: 5.0
AirQualityIndex-Min: 1.0
Carbon Monoxide-Max: 4058.84
Carbon Monoxide-Min: 233.65
PM2.5-Max: 448.83
PM2.5-Min: 8.24
PM10-Max: 486.19
PM10-Min: 8.26


In [120]:
# Task 4: Identify outliers and bad data
# Checking unique values for some of the pollution parameters
# AQI, Carbon Monoxide, PM2.5, PM10
# Additionally they should not be negative
print("AirQualityIndex-Max:",df_api['AirQualityIndex'].max())
print("AirQualityIndex-Min:",df_api['AirQualityIndex'].min())
print("Carbon Monoxide-Max:",df_api['CarbonMonoxide'].max())
print("Carbon Monoxide-Min:",df_api['CarbonMonoxide'].min())
print("PM2.5-Max:",df_api['PM2.5'].max())
print("PM2.5-Min:",df_api['PM2.5'].min())
print("PM10-Max:",df_api['PM10'].max())
print("PM10-Min:",df_api['PM10'].min())
# None of the values are negative, so should be good.

AirQualityIndex-Max: 5.0
AirQualityIndex-Min: 1.0
Carbon Monoxide-Max: 4058.84
Carbon Monoxide-Min: 233.65
PM2.5-Max: 448.83
PM2.5-Min: 8.24
PM10-Max: 486.19
PM10-Min: 8.26


In [ ]:
# There is hardle much I can do with the data returned from API
# However Carried out the below 5 tasks, which are marked in the solutions
# Task 1: Format data into a more readable format
# Task 2: Replace Headers
# Task 3: Find if any duplicates
# Checking the Combination of PC Name and State Name
# Task 4: Identify outliers and bad data
# Checking unique values for some of the pollution parameters
# Task 5: Fix casing or inconsistent values
# Inconsistent Values have been taken care